In [41]:
intcode_program = [3,8,1001,8,10,8,105,1,0,0,21,34,47,72,93,110,191,272,353,434,99999,3,9,102,3,9,9,1001,9,3,9,4,9,99,3,9,102,4,9,9,1001,9,4,9,4,9,99,3,9,101,3,9,9,1002,9,3,9,1001,9,2,9,1002,9,2,9,101,4,9,9,4,9,99,3,9,1002,9,3,9,101,5,9,9,102,4,9,9,1001,9,4,9,4,9,99,3,9,101,3,9,9,102,4,9,9,1001,9,3,9,4,9,99,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,99,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,99,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,99]

In [116]:
from collections import deque
intcode_outputs = {5: deque(), 6: deque(), 7: deque(), 8: deque(), 9: deque()}

def get_val(arr, position, parameter=0):
    if parameter:
        return arr[position]
    else:
        return arr[arr[position]]

def intcode_parser(arr, pos, phase_setting, dependency, first_run, first_producer, consume, produce):
    op = arr[pos]
    parameters = (0, 0, 0)
    if op > 99:
        parameters = ((op // 100) % 10, (op // 1000) % 10, (op // 10000) % 10)
        op = op % 100
        #print(parameters)
        
    if op == 99:
        return True
    elif op == 1:
        arr[arr[pos + 3]] = get_val(arr, pos + 1, parameters[0]) + get_val(arr, pos + 2, parameters[1])
        return intcode_parser(arr, pos + 4, phase_setting, dependency, first_run, first_producer, consume, produce)
    elif op == 2:
        arr[arr[pos + 3]] = get_val(arr, pos + 1, parameters[0]) * get_val(arr, pos + 2, parameters[1])
        return intcode_parser(arr, pos + 4, phase_setting, dependency, first_run, first_producer, consume, produce)
    elif op == 3:
        if first_run:
            arr[arr[pos + 1]] = phase_setting
            return intcode_parser(arr, pos + 2, phase_setting, dependency, False, first_producer, consume, produce)
        elif not first_run and first_producer:
            arr[arr[pos + 1]] = 0
            return intcode_parser(arr, pos + 2, phase_setting, dependency, False, False, consume, produce)
        else:
            with consume:
                while len(intcode_outputs[dependency]) == 0:
                    consume.wait()
                arr[arr[pos + 1]] = intcode_outputs[dependency].popleft()
            return intcode_parser(arr, pos + 2, phase_setting, dependency, first_run, first_producer, consume, produce)
    elif op == 4:
        with produce:
            intcode_outputs[phase_setting].append(get_val(arr, pos + 1, parameters[0]))
            produce.notifyAll()
        return intcode_parser(arr, pos + 2, phase_setting, dependency, first_run, first_producer, consume, produce)
    elif op == 5:
        if get_val(arr, pos + 1, parameters[0]) != 0:
            return intcode_parser(arr, get_val(arr, pos + 2, parameters[1]), phase_setting, dependency, first_run, first_producer, consume, produce)
        else:
            return intcode_parser(arr, pos + 3, phase_setting, dependency, first_run, first_producer, consume, produce)
    elif op == 6:
        if get_val(arr, pos + 1, parameters[0]) == 0:
            return intcode_parser(arr, get_val(arr, pos + 2, parameters[1]), phase_setting, dependency, first_run, first_producer, consume, produce)
        else:
            return intcode_parser(arr, pos + 3, phase_setting, dependency, first_run, first_producer, consume, produce)
    elif op == 7:
        if get_val(arr, pos + 1, parameters[0]) < get_val(arr, pos + 2, parameters[1]):
            arr[arr[pos + 3]] = 1
        else:
            arr[arr[pos + 3]] = 0
        return intcode_parser(arr, pos + 4, phase_setting, dependency, first_run, first_producer, consume, produce)
    elif op == 8:
        if get_val(arr, pos + 1, parameters[0]) == get_val(arr, pos + 2, parameters[1]):
            arr[arr[pos + 3]] = 1
        else:
            arr[arr[pos + 3]] = 0
        return intcode_parser(arr, pos + 4, phase_setting, dependency, first_run, first_producer, consume, produce)
    else:
        print("Uknown op code " + str(op) + "at position " + str(pos))
        return False

In [125]:
from collections import Counter
from copy import deepcopy
import threading
import time
import logging

logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-9s) %(message)s',)


intcode_program = [3,8,1001,8,10,8,105,1,0,0,21,34,47,72,93,110,191,272,353,434,99999,3,9,102,3,9,9,1001,9,3,9,4,9,99,3,9,102,4,9,9,1001,9,4,9,4,9,99,3,9,101,3,9,9,1002,9,3,9,1001,9,2,9,1002,9,2,9,101,4,9,9,4,9,99,3,9,1002,9,3,9,101,5,9,9,102,4,9,9,1001,9,4,9,4,9,99,3,9,101,3,9,9,102,4,9,9,1001,9,3,9,4,9,99,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,99,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,99,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,99]


phase_settings = [5, 6, 7, 8, 9]
max_power = 0
max_configuration = (0, 0, 0, 0, 0)
for i in phase_settings:
    for j in phase_settings:
        for k in phase_settings:
            for l in phase_settings:
                for m in phase_settings:
                    configuration = [m, l, k, j, i]
                    if all([i == 1 for i in Counter(configuration).values()]):
                        
                        condition_ml = threading.Condition()
                        condition_lk = threading.Condition()
                        condition_kj = threading.Condition()
                        condition_ji = threading.Condition()
                        condition_im = threading.Condition()


                        prgrm_m = threading.Thread(name='program_m', target=intcode_parser, 
                                                args=(deepcopy(intcode_program), 0, m, i, True, True, condition_im, condition_ml,))
                        prgrm_m.start()

                        prgrm_l = threading.Thread(name='program_l', target=intcode_parser, 
                                                args=(deepcopy(intcode_program), 0, l, m, True, False, condition_ml, condition_lk,))
                        prgrm_l.start()

                        prgrm_k = threading.Thread(name='program_l', target=intcode_parser, 
                                                args=(deepcopy(intcode_program), 0, k, l, True, False, condition_lk, condition_kj,))
                        prgrm_k.start()

                        prgrm_j = threading.Thread(name='program_j', target=intcode_parser, 
                                                args=(deepcopy(intcode_program), 0, j, k, True, False, condition_kj, condition_ji,))
                        prgrm_j.start()

                        prgrm_i = threading.Thread(name='program_i', target=intcode_parser, 
                                                args=(deepcopy(intcode_program), 0, i, j, True, False, condition_ji, condition_im,))
                        prgrm_i.start()


                        prgrm_m.join()
                        prgrm_l.join()
                        prgrm_k.join()
                        prgrm_j.join()
                        prgrm_i.join()
                        last_e = intcode_outputs[i].popleft()
                        if last_e > max_power:
                            max_power = last_e
                            max_configuration = tuple(configuration)

                        intcode_outputs = {5: deque(), 6: deque(), 7: deque(), 8: deque(), 9: deque()}


In [126]:
print(max_power)
print(max_configuration)

27561242
(7, 9, 5, 8, 6)


4257

In [20]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
from intcode import *

from collections import Counter
from copy import deepcopy
import threading
import time
import logging

logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-9s) %(message)s',)


intcode_program = [3,8,1001,8,10,8,105,1,0,0,21,34,47,72,93,110,191,272,353,434,99999,3,9,102,3,9,9,1001,9,3,9,4,9,99,3,9,102,4,9,9,1001,9,4,9,4,9,99,3,9,101,3,9,9,1002,9,3,9,1001,9,2,9,1002,9,2,9,101,4,9,9,4,9,99,3,9,1002,9,3,9,101,5,9,9,102,4,9,9,1001,9,4,9,4,9,99,3,9,101,3,9,9,102,4,9,9,1001,9,3,9,4,9,99,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,99,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,99,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,99]

intcode_outputs = {5: deque(), 6: deque(), 7: deque(), 8: deque(), 9: deque()}
phase_settings = [5, 6, 7, 8, 9]
max_power = 0
max_configuration = (0, 0, 0, 0, 0)
for i in phase_settings:
    for j in phase_settings:
        for k in phase_settings:
            for l in phase_settings:
                for m in phase_settings:
                    configuration = [m, l, k, j, i]
                    if all([i == 1 for i in Counter(configuration).values()]):
                        
                        condition_ml = threading.Condition()
                        condition_lk = threading.Condition()
                        condition_kj = threading.Condition()
                        condition_ji = threading.Condition()
                        condition_im = threading.Condition()

                        prgrm_m = threading.Thread(name='program_m', target=init_intcode_run, 
                                                args=(deepcopy(intcode_program), 0, m, i, intcode_outputs, True, True, condition_im, condition_ml,))
                        prgrm_m.start()
            
                        prgrm_l = threading.Thread(name='program_l', target=init_intcode_run, 
                                                args=(deepcopy(intcode_program), 0, l, m, intcode_outputs, True, False, condition_ml, condition_lk,))
                        prgrm_l.start()

                        prgrm_k = threading.Thread(name='program_l', target=init_intcode_run, 
                                                args=(deepcopy(intcode_program), 0, k, l, intcode_outputs, True, False, condition_lk, condition_kj,))
                        prgrm_k.start()

                        prgrm_j = threading.Thread(name='program_j', target=init_intcode_run, 
                                                args=(deepcopy(intcode_program), 0, j, k, intcode_outputs, True, False, condition_kj, condition_ji,))
                        prgrm_j.start()

                        prgrm_i = threading.Thread(name='program_i', target=init_intcode_run, 
                                                args=(deepcopy(intcode_program), 0, i, j, intcode_outputs, True, False, condition_ji, condition_im,))
                        prgrm_i.start()


                        prgrm_m.join()
                        prgrm_l.join()
                        prgrm_k.join()
                        prgrm_j.join()
                        prgrm_i.join()
                        last_e = intcode_outputs[i].popleft()
                        if last_e > max_power:
                            max_power = last_e
                            max_configuration = tuple(configuration)
                        intcode_outputs = {5: deque(), 6: deque(), 7: deque(), 8: deque(), 9: deque()}



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
print(max_power)
print(max_configuration)

27561242
(7, 9, 5, 8, 6)
